In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import Necessary libraries
import numpy as np 
import pandas as pd 

# Import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

# Import Sampler libraries
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline

# Import MRMR (Minimum Redundancy Maximum Relevance) feature selection
from skfeature.function.information_theoretical_based import MRMR


In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

In [3]:
# Read the data
df = pd.read_csv("diabetes_prediction_dataset.csv")

In [4]:
# Handle duplicates
df = df.drop_duplicates()

In [5]:
# Remove unnecessary values [0.00195%]
df = df[df['gender'] != 'Other']

In [6]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'


In [7]:
df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)

In [8]:
df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)

# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df


In [9]:
data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')


In [10]:
# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)


In [11]:
# Feature Selection (MRMR)
X = data.drop('diabetes', axis=1)
y = data['diabetes']
feat_idx = MRMR.mrmr(X.values, y.values, n_selected_features=10)
selected_features = X.columns[feat_idx].tolist()  # Convert to list

In [12]:
data = data[selected_features + ['diabetes']]  # Ensure diabetes column is included

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), data.drop('diabetes', axis=1).select_dtypes(include='number').columns),
        ('cat', OneHotEncoder(), data.drop('diabetes', axis=1).select_dtypes(include='object').columns)
    ])

In [14]:
# Model Pipeline
clf = imbPipeline(steps=[('preprocessor', preprocessor),
                         ('over', over),
                         ('under', under),
                         ('classifier', DecisionTreeClassifier(
                             max_depth=8, 
                             min_samples_split=0.2, 
                             min_samples_leaf=2, 
                             criterion='gini'
                         ))])


In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Train the model
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['HbA1c_level', 'bmi', 'heart_disease', 'age', 'hypertension',
       'blood_glucose_level'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index([], dtype='object'))])),
                ('over', SMOTE(sampling_strategy=0.1)),
                ('under', RandomUnderSampler(sampling_strategy=0.5)),
                ('classifier',
                 DecisionTreeClassifier(max_depth=8, min_samples_leaf=2,
                                        min_samples_split=0.2))])

In [17]:
# Predict on the test set
y_pred = clf.predict(X_test)

In [18]:
# Evaluate the model
print("Model Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Model Accuracy:  0.9692603765733901
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     17525
           1       1.00      0.65      0.79      1701

    accuracy                           0.97     19226
   macro avg       0.98      0.83      0.89     19226
weighted avg       0.97      0.97      0.97     19226

